In [10]:
import zipfile

folder='/Users/swaroop/Downloads/SrDA-Dataset-Assignment (1).zip'
destination='/Users/swaroop/Downloads'

with zipfile.ZipFile(folder,'r') as z:
    z.extractall(destination)
    top_level = z.namelist()[0].split("/")[0]

print()

Sr DA - Dataset - Assignment 


In [151]:
import pandas as pd
import numpy as np
import os
File=[]
path='/Users/swaroop/Downloads/Sr DA - Dataset - Assignment '
for file in os.listdir(path):
    if file.endswith('csv'):
        File.append(file)

cutomer_df=pd.read_csv(path+'/'+File[0])
Products_df=pd.read_csv(path+'/'+File[1])
trxn_df=pd.read_csv(path+'/'+File[2])
session_df=pd.read_csv(path+'/'+File[3])
Support_tickets_df=pd.read_csv(path+'/'+File[4])

#### Customer 

In [156]:
### handling null values 
cutomer_df=cutomer_df.fillna('unknown')

In [158]:
from datetime import datetime

def fix_date(x):
    for fmt in ("%Y-%m-%d", "%m/%d/%Y","%d-%m-%Y"):
        try:
            return datetime.strptime(x, fmt).strftime("%Y-%m-%d")
        except:
            pass
    return None  # or x

cutomer_df["signup_date"] = cutomer_df["signup_date"].apply(fix_date)

In [162]:
cutomer_df.to_csv('customer_transformed.csv')

In [160]:
cutomer_df.head(10)

,customer_id,signup_date,country,email_verified,referred_by
0,C1000,2024-08-02,DE,True,ad_campaign
1,C1001,2021-08-17,IN,True,unknown
2,C1002,2021-02-21,US,unknown,unknown
3,C1003,2022-07-18,unknown,False,ad_campaign
4,C1004,2022-05-17,AU,True,unknown
5,C1005,2022-04-03,unknown,False,affiliate
6,C1006,2021-10-13,BR,unknown,affiliate
7,C1007,2021-07-29,GB,unknown,unknown
8,C1008,2024-10-17,BR,unknown,unknown
9,C1009,2024-01-22,IN,unknown,affiliate


#### Products

In [91]:
Products_df.groupby(['product_id','product_name','category']).count().sort_values(by='list_price',ascending=False).head()

,,,list_price
product_id,product_name,category,
P200,Product_0,Accessories,1
P237,Product_37,Electronics,1
P227,Product_27,Accessories,1
P228,Product_28,Accessories,1
P229,Product_29,Accessories,1


In [144]:
# Products_df= Products_df[Products_df['list_price']>0]
# Products_df.to_csv('product_transformed')
Products_df[Products_df['list_price']<0]

,product_id,product_name,category,list_price
44,P244,Product_44,Accessories,-10.0
46,P246,Product_46,Electronics,-5.0


#### Sessions

In [183]:
Sessions_df=session_df.copy()
Sessions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   session_id     2500 non-null   object 
 1   customer_id    2476 non-null   object 
 2   session_start  2500 non-null   object 
 3   session_end    2368 non-null   object 
 4   pages_viewed   2169 non-null   float64
 5   device         2500 non-null   object 
dtypes: float64(1), object(5)
memory usage: 117.3+ KB


In [184]:
pd.set_option('display.max_rows',None)
Sessions_df[(Sessions_df['session_end'].isnull()) & (Sessions_df['pages_viewed'].isnull())
].head(5)

,session_id,customer_id,session_start,session_end,pages_viewed,device
247,S70247,C1013,20/04/2023 10:24,NaN,NaN,web
347,S70347,C1228,04/08/2023 02:31,NaN,NaN,web
444,S70444,C1770,02/04/2024 20:45,NaN,NaN,android
456,S70456,C1741,19/02/2024 20:27,NaN,NaN,web
1022,S71022,C1706,07/09/2024 06:57,NaN,NaN,android app


In [185]:
formats = [ "%Y-%m-%d %H:%M:%S",
            "%m/%d/%Y %H:%M:%S",
            "%d/%m/%Y %H:%M:%S",
            "%d/%m/%Y %H:%M"
          ]

def fix_date(x):
    for fmt in formats:
        try:
            return datetime.strptime(x, fmt).strftime("%Y-%m-%d %H:%M:%S")
        except:
            pass
    return None  # or x

In [186]:
Sessions_df['session_start']=Sessions_df['session_start'].apply(fix_date)
Sessions_df['session_end']=Sessions_df['session_end'].apply(fix_date)

In [187]:
indexes=Sessions_df[Sessions_df['session_start']> Sessions_df['session_end']].index
print('# sessions with start_ts>end_ts',len(indexes))

Sessions_df.drop(indexes,inplace=True)

# sessions with start_ts>end_ts 42


In [188]:
#### replace null customer_id will unknow
Sessions_df['customer_id'].fillna('unknown',inplace=True)
Sessions_df['pages_viewed'].fillna(0,inplace=True)

In [189]:
Sessions_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2458 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   session_id     2458 non-null   object 
 1   customer_id    2458 non-null   object 
 2   session_start  2458 non-null   object 
 3   session_end    2326 non-null   object 
 4   pages_viewed   2458 non-null   float64
 5   device         2458 non-null   object 
dtypes: float64(1), object(5)
memory usage: 134.4+ KB


In [190]:
Sessions_df.to_csv('session_transformed')

In [191]:
Sessions_df['session_start_year']=Sessions_df['session_start'].apply(pd.to_datetime,errors='coerce').dt.year

In [214]:
Sessions_df['week_start'] = Sessions_df['session_start'].apply(pd.to_datetime ,errors='coerce').dt.to_period('W').dt.to_timestamp()
Sessions_df.head(5)

,session_id,customer_id,session_start,session_end,pages_viewed,device,session_start_year,month_start,week_start
0,S70000,C1603,2025-01-13 13:47:00,2025-01-13 15:52:00,4.0,web,2025,2025-01-01,2025-01-13
1,S70001,C1180,2024-11-25 02:11:00,2024-11-25 02:27:00,3.0,ios,2024,2024-11-01,2024-11-25
2,S70002,C1432,2024-11-27 12:05:00,2024-11-27 14:52:00,0.0,mobile,2024,2024-11-01,2024-11-25
3,S70003,C1429,2023-09-24 19:44:00,2023-09-24 19:52:00,0.0,desktop,2023,2023-09-01,2023-09-18
4,S70004,C1130,2024-10-16 19:13:00,2024-10-16 21:29:00,1.0,web,2024,2024-10-01,2024-10-14


In [218]:
df= Sessions_df.groupby('week_start')['session_id'].count().reset_index()
df

,week_start,session_id
0,2022-12-26,4
1,2023-01-02,34
2,2023-01-09,22
3,2023-01-16,26
4,2023-01-23,15
5,2023-01-30,19
6,2023-02-06,19
7,2023-02-13,21
8,2023-02-20,26
9,2023-02-27,19


#### Transaction

In [92]:
Transactions_df=trxn_df.copy()
Transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3020 entries, 0 to 3019
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  3020 non-null   object 
 1   customer_id     3020 non-null   object 
 2   product_id      3020 non-null   object 
 3   quantity        3020 non-null   int64  
 4   unit_price      2874 non-null   float64
 5   total_amount    2874 non-null   float64
 6   created_at      3020 non-null   object 
 7   payment_status  2506 non-null   object 
 8   order_source    2499 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 212.5+ KB


In [93]:
Transactions_df[Transactions_df['transaction_id']=='T50002']

,transaction_id,customer_id,product_id,quantity,unit_price,total_amount,created_at,payment_status,order_source
2,T50002,C1271,P209,5,55.34,276.7,10/01/2023 08:04 AM,charged_back,web


In [60]:
# Transactions_df['created_at']=Transactions_df['created_at'].str.replace('AM','').str.replace('PM','')

In [94]:
from datetime import datetime

formats = [ "%Y-%m-%d %H:%M:%S",
            "%d-%m-%Y %H:%M:%S",
            "%d-%m-%Y %H:%M",
            "%m/%d/%Y %H:%M:%S",
            "%d/%m/%Y %H:%M:%S",
            "%d/%m/%Y %H:%M",
            "%m/%d/%Y %H:%M",
           "%m/%d/%Y %I:%M %p",   # 10/01/2023 08:04 AM
           "%m-%d-%Y %I:%M %p"   # 10-01-2023 08:04 AM
          ]

def fix_date(x):
    for fmt in formats:
        try:
            return datetime.strptime(x, fmt).strftime("%Y-%m-%d %H:%M:%S")
        except:
            pass
    return None  # or x

In [95]:
pd.set_option('display.max_rows',None)
Transactions_df['created_at']=Transactions_df['created_at'].apply(fix_date)

In [96]:
cols = ['unit_price', 'total_amount']
Transactions_df[cols] = Transactions_df[cols].apply(pd.to_numeric, errors='coerce').mask(lambda x: x < 0, np.nan)

In [106]:
Transactions_df['unit_price']= Transactions_df['unit_price'].fillna(
    Transactions_df.groupby('product_id')['unit_price'].transform('mean')
)

In [111]:
Transactions_df[['payment_status','order_source']]=Transactions_df[['payment_status','order_source']].fillna('unknown')

In [113]:
Transactions_df['Total']=np.round(Transactions_df['quantity']*Transactions_df['unit_price'],2)
Transactions_df

,transaction_id,customer_id,product_id,quantity,unit_price,total_amount,created_at,payment_status,order_source,Total
0,T50000,C1747,P209,2,51.710000,103.42,2025-12-08 09:35:00,refunded,store,103.42
1,T50001,C1553,P210,2,93.560000,187.12,2023-08-13 14:19:00,paid,web,187.12
2,T50002,C1271,P209,5,55.340000,276.70,2023-10-01 08:04:00,charged_back,web,276.70
3,T50003,C1676,P211,5,176.600000,883.00,2025-02-26 15:10:00,PAID,partner,883.00
4,T50004,C1288,P249,5,150.560000,752.80,2023-08-13 08:22:00,unknown,unknown,752.80
5,T50005,C1304,P245,1,128.040000,128.04,2024-03-14 10:50:00,refunded,web,128.04
6,T50006,C1243,P203,2,94.780000,189.56,2023-07-21 16:52:00,refunded,web,189.56
7,T50007,C1098,P226,1,47.250000,47.25,2024-02-03 18:16:00,pending,unknown,47.25
8,T50008,C1050,P219,3,74.090000,222.27,2024-04-04 18:48:00,unknown,unknown,222.27
9,T50009,C1402,P219,1,84.210000,84.21,2023-01-12 05:22:00,charged_back,store,84.21


In [116]:
Transactions_df.drop(columns=['total_amount'],inplace=True)
Transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3020 entries, 0 to 3019
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  3020 non-null   object 
 1   customer_id     3020 non-null   object 
 2   product_id      3020 non-null   object 
 3   quantity        3020 non-null   int64  
 4   unit_price      3020 non-null   float64
 5   created_at      3020 non-null   object 
 6   payment_status  3020 non-null   object 
 7   order_source    3020 non-null   object 
 8   Total           3020 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 212.5+ KB


In [167]:
Transactions_df=Transactions_df[Transactions_df['created_at'].apply(pd.to_datetime, errors='coerce').dt.year <= 2025]

In [171]:
Transactions_df['month']=Transactions_df['created_at'].apply(pd.to_datetime, errors='coerce').dt.month

/var/folders/wn/nv99sn0j19vc0y9rx_ltp_v40000gn/T/ipykernel_2403/3999203662.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Transactions_df['month']=Transactions_df['created_at'].apply(pd.to_datetime, errors='coerce').dt.month


In [196]:
Transactions_df.groupby(['order_source']).agg({

    'transaction_id':'count',
    'Total': 'sum',
    'quantity': 'sum'

}).reset_index()

,order_source,transaction_id,Total,quantity
0,mobile,505,112007.35,1057
1,partner,508,111993.92,1112
2,store,526,115750.63,1144
3,unknown,519,112566.55,1153
4,web,931,217656.72,2107


In [168]:
Transactions_df.to_csv('transactions_df_transformed.csv')

In [202]:
Transactions_df[(Transactions_df['payment_status']=='paid') | (Transactions_df['payment_status']=='PAID')]['customer_id'].nunique()

563

#### support tickets 

In [120]:
support_df=Support_tickets_df.copy()
support_df.head(5)

,ticket_id,customer_id,created_at,resolved_at,topic,resolution_status
0,TK9000,C1015,2023-01-12,2023-01-19,other,closed
1,TK9001,C1403,2023-12-25,2024-01-08,account,resolved
2,TK9002,C1204,02-03-2023,2023-03-02,delivery,closed
3,TK9003,C1129,20-12-2024,2024-12-20,technical,closed
4,TK9004,C1759,2024-07-19,2024-08-03,delivery,resolved


In [130]:
from datetime import datetime

format=[
    "%Y-%m-%d",
    "%d-%m-%Y",
    "%m-%d-%Y",
]

def formating(x):
    for fmt in format:
        try:
            return datetime.strptime(x, fmt).strftime("%Y-%m-%d")
    
        except:
            pass
    return None 

In [132]:
support_df['created_at']=support_df['created_at'].apply(formating)

In [134]:
support_df['resolved_at']=support_df['resolved_at'].apply(formating)

In [137]:
support_df[['created_at','resolved_at']]=support_df[['created_at','resolved_at']].apply(pd.to_datetime,errors='coerce')

In [142]:
support_df.to_csv('support_ticket_transformed.csv')